In [8]:
import os
import numpy as np 
import pandas as pd 
from cv2 import cv2 as cv
import matplotlib
import matplotlib.pyplot as plt 

<center><font face="宋体" size=15> 机器学习报告</font></center>
<center> 17081519 沈鸿辉 </center>
<center> 注：为了更好的显示数学公式、代码和代码运行结果，本报告由jupyter文件导出 </center>

## 【实验项目】实验一：基于kNN的验证码识别

## 【实验目的】
+ 学习掌握kNN算法的数学原理
+ 学习掌握python-opencv处理图像的基本方法
+ 学习使用sklearn现有的kNN工具做模型预测
+ 自己尝试实现基于kNN算法的分类器，实现模型预测

## 【实验原理】

### kNN算法原理
kNN是通过测量不同特征值之间的距离进行分类。它的思路是：如果一个样本在特征空间中的k个最相似(即特征空间中最邻近)的样本中的大多数属于某一个类别，则该样本也属于这个类别，其中K通常是不大于20的整数。kNN算法中，所选择的邻居都是已经正确分类的对象。该方法在定类决策上只依据最邻近的一个或者几个样本的类别来决定待分样本所属的类别。K-近邻算法的具体流程如下：
+ 首先，计算新样本与训练样本之间的距离，找到距离最近的K个邻居；
+ 然后，根据这些邻居所属的类别来判定新样本的类别，如果它们都属于同一个类别，那么新样本也属于这个类；
+ 否则，对每个候选类别进行评分，按照某种规则确定新样本的类别。
在kNN中，通过计算对象间距离来作为各个对象之间的非相似性指标，避免了对象之间的匹配问题，在这里距离一般使用欧氏距离或曼哈顿距离：
  - n维欧式距离公式：
    $$
    d(x, y):=\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}+\cdots+\left(x_{n}-y_{n}\right)^{2}}=\sqrt{\sum_{i=1}^{n}\left(x_{i}-y_{i}\right)^{2}}
    $$
  - 曼哈顿距离也称为城市街区距离(City Block distance)：   
      两个n维向量 $\vec{a}=\left(x_{1}, x_{2}, \cdots, x_{n}\right)$ 和 $\vec{b}=\left(y_{1}, y_{2}, \cdots, y_{n}\right)$ 之间的曼哈顿距离：
    $$
    d(\vec{a}, \vec{b})=d(x, y)=\sum_{i=1}^{n}\left|x_{i}-y_{i}\right|
    $$
+ 本项目采用欧式距离  
+ [其他距离公式](https://blog.csdn.net/Kevin_cc98/article/details/73742037)

### openCV轮廓识别函数的使用
在图像处理中，我们经常需要检测具有相同颜色的像素团（blob）。 这些像素团周围的边界被称为轮廓 （contour）。 OpenCV有一个内置的findContours()函数，可以用来检测这些连续的区域。

    findContours(image, mode, method, contours=None, hierarchy=None, offset=None)
+ mode：轮廓的查找模式，一般使用cv2.RETR_TREE，表示提取所有的轮廓并建立轮廓间的层级。
+ method：轮廓的近似方法。比如对于一条直线，我们可以存储该直线的所有像素点，也可以只存储起点和终点。使用cv2.CHAIN_APPROX_SIMPLE就表示用尽可能少的像素点表示轮廓。
+ contours、hierarchy同时也为函数返回值，offset为偏移
+ 参考：[C++ findContours函数参数详解](https://blog.csdn.net/dcrmg/article/details/51987348)

## 【设计内容】
主要的设计是下面的函数体，用于实现验证码训练集生成、验证码单字符分割、kNN分类器识别

### 图像处理函数主体，生成指定格式的图像

In [9]:
# 为了方便图片直接显示在jupyter中，cv的imshow不能直接在jupyter中显示
# 为了用matplotlib显示（ 不能用plt.show()，要用plt.imshow() ）
# 由于CV的通道是BGR顺序，而matpotlib是 RGB顺序，这里要做通道转换
# 方法一
def bgr2rgb_v2(img):
    # 用cv自带的分割和合并函数
    B,G,R = cv.split(img)
    return cv.merge()
# 方法二
def bgr2rgb(img):
    # 直接用python切片特性，[start: end: step], 这里start end为空，则默认遍历全部，step为-1则倒序遍历
    return img[:, :, ::-1]

In [10]:
def genNeedImg(img_path, img_type='binary', binary_therhold=127, 
               binary_revese=False, size=None, save=False, path='./'):
    '''
    用于生成指定大小的灰度图或二值图, img_path为图像路径
    type为标志转换类型，默认为binary，可选的值为binary或gray
    binary_therhold为二值图划分阈值，默认127（即大于127的像素设置为255，否则置0）
    binary_revese默认为False，True时黑白颠倒（即大于127的像素设置为0，否则置255）
    size为tuple类型，用于指定生成图像的尺寸, 如：(512,512)，默认为None表示输出原图像尺寸
    save为保存标志，默认为False，为true时将生成的图保存到path(默认为当前文件夹)
    '''
    img_raw = cv.imread(img_path)
    if size != None: # 调整图像尺寸
        img_raw= cv.resize(img_raw,size)
    img_gray = cv.cvtColor(img_raw,cv.COLOR_RGB2GRAY) # 转换颜色空间为灰度
    img_name = img_path[9:].split('.')[0] # 获取图像原始名称
    if img_type == 'gray': # 生成灰度图
        if save:
            cv.imwrite(os.path.join(path,'{}_gray.bmp'.format(img_name)),img_gray)
        else:
            return img_gray
        print('Gray image generated!')
    else: # 生成二值图
        if binary_revese:
            ret, img_binary = cv.threshold(img_gray,binary_therhold,255,cv.THRESH_BINARY_INV) #反二进制阈值化
        else:
            ret, img_binary = cv.threshold(img_gray,binary_therhold,255,cv.THRESH_BINARY)# 二进制阈值化
        if save:
            cv.imwrite(os.path.join(path,'{}_binary.bmp'.format(img_name)),img_binary)
        else:
            return img_binary
        print('Binary image generated!')
        print('threshold:{}'.format(ret)) # 输出转换阈值

### 训练集生成函数主体

In [11]:
def captcha_generator(character_set:str, font_set=None, save_path='./', size=(150,60),
                         style=((255, 0, 0), (255, 255, 255), 40, False, (32,127)), captcha_len=4):
    '''
    R.G.的验证码生成器
    character_set: 用于生成验证码的字符集，字符集以str类型传入
    font_set: 字体集，str类型，本机的字体路径，如'path/to/font.ttf'
    save_path: 验证码图片的保存位置，默认为当前文件夹
    size: 生成验证码图片的尺寸，tuple类型，格式：（长，宽)，默认（150, 60）
    style: 生成验证码的样式，tuple类型，格式：(text_color, background_color, text_size, rnd_background, fill_range), 
            _color均为三元tuple, text_size为int, rnd_background为bool用于控制是否把背景随机填色，默认不随机填色
            fill_range: tuple类型，每个像素的颜色范围0~255，默认(32,127)【fill_range仅当rnd_background=True时起作用】
    captcha_len: 要生成的验证码长度，即验证码字符个数
    '''
    # captcha_text = ''.join(random.sample(character_set, captcha_len)) # 这样生成的验证码中不会出现相同字符
    # captcha_text = ''.join([ random.choice(character_set) for i in range(captcha_len)]) # 验证码中可能出现相同字符
    captcha_text = character_set
    background_img = Image.new('RGB', size, style[1]) # 创建一个Image对象，new(mode, size, color=0) 
    my_font = ImageFont.truetype(font=font_set, size=style[2]) # 创建字体对象
    # 注意一点，之前用captcha库时候，font传入的是list类型，而这里传入的要是一个str类型
    if style[3]:
        background_img = background_rnd_fill(background_img, fill_range=style[4])
    captcha_img = draw_text(background_img, captcha_text, font=my_font, fill=style[0])
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    captcha_save_path = os.path.join(save_path, '{}.png'.format(captcha_text))
    i = 0
    while True:
        i += 1
        if os.path.exists(captcha_save_path): # 已经有样本，则在正确标签后面加_i, i标记重复次数
            captcha_save_path = os.path.join(save_path, '{}_{}.png'.format(captcha_text, i))
        else: # 不存在重名路径，则跳出
            break
    captcha_img.save(captcha_save_path, 'png')
    print('Captcha {} generated!'.format(captcha_text))
    return captcha_text

### 验证码单字符分割函数主体

In [12]:
def captcha_character_detach(captcha_img_path, characters_save_path='./', captcha_len=4):
    '''
    验证码分割函数，用cv的findcontours()识别字符轮廓，然后按轮廓框定矩形，分割出每个字符
    '''
    captcha_img_basename = os.path.basename(captcha_img_path) # 从路径中提取带后缀文件名，如 '0415.png'
    captcha_text = os.path.splitext(captcha_img_basename)[0] # ['0415', 'png']
    img_gray = cv.imread(captcha_img_path, cv.IMREAD_GRAYSCALE) # 灰度图读入
    img_binary = genNeedImg(captcha_img_path, img_type='binary', binary_therhold=127, binary_reverse=True) # 直接调用genNeedImg生成二值图
    contours, hierarchy = cv.findContours(img_binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) # 划分字符轮廓
    # 由于直接划分的轮廓太多了（我换了字体后好像不会划分很多），这里考虑记录每个每个轮廓的数据，然后取 wxh （长x宽，即面积）的top4
    boundings = [cv.boundingRect(contour) for contour in contours] # 获取每个轮廓的信息，(x,y,width,height) x,y为轮廓最左上角坐标
    boundings.sort(key=lambda tuple_x: tuple_x[2]*tuple_x[3], reverse=True) # lamdba传入的就是计算每个轮廓的面积，然后按面积大小降序排序
    if len(boundings) < captcha_len: # 获取到的轮廓小于4，则说明没有把4个字符都区分开来
        print('Bondings less then 4, captcha discarded!')
        return # 直接结束，丢弃这个验证码样本
    '''
    # 下面开始画矩形分割框，这部分其实用不到，只是为了调试看画的样子
    # -----------------------------------------------------------------------------------------
    temp_img = cv.imread(captcha_img_path, cv.IMREAD_UNCHANGED) # 以原始格式读入图片
    temp_img = bgr2rgb(temp_img) # 通道转换
    for bounding in boundings[:4]: # 取面积最大的前4个轮廓
        x, y, width, height = bounding
        img_addBox = cv.rectangle(temp_img, (x,y), (x + width, y + height), (0, 255, 0), 1)
    plt.imshow(img_addBox, cmap='gray')
    # ------------------------------------------------------------------------------------------
    '''
    boundings_save = sorted(boundings[:captcha_lencaptcha_len], key=lambda tuple_x: tuple_x[0]) # 按轮廓的x坐标大小排序，tuple_x=(x,y,width,height) 
    character_splited = []
    for character_bounding, character_text in zip(boundings_save, captcha_text):
        x, y, width, height = character_bounding
        margin = 2 # 提取单个字符的时候，在获取的轮廓拓宽margin个像素，因为findContours()的轮廓可能很紧凑
        character_img = img_gray[y - margin:y + height + margin, x - margin:x + width + margin]
        if not os.path.exists(characters_save_path): # 如果要保存的路径不存在就创建该路径目录
            os.makedirs(characters_save_path)
        character_path = os.path.join(characters_save_path, '{}_0.png'.format(character_text))
        i = 0
        while True:
            i += 1
            if os.path.exists(character_path): # 该字符已经有样本，则在正确标签后面加_i, i标记重复次数
                character_path = os.path.join(characters_save_path, '{}_{}.png'.format(character_text, i))
            else: # 不存在重名路径，则跳出
                break
        cv.imwrite(character_path, character_img)
        character_splited.append(character_img)
    print('Character detached from captcha, character has been saved at {}'.format(characters_save_path))
    return character_splited

### kNN分类识别器

In [13]:
def judger(k_prediction):
    '''
    判决器，k_prediction是有权重的，索引最小代表该预测值和实际值相似度最大
    但是，可能出现如：k=5，k_prediction=（7, 1, 2, 2, 2），这样的情况，我定预测结果为2
    也就是，重复预测值个数 >= 预测值总数的2/3，则取该重复预测值
    '''
    k = len(k_prediction)
    tmp = list(k_prediction)
    [tmp.remove(i) for i in list(set(tmp))]
    duplication = set(tmp) # 找出预测值中重复元素
    if len(duplication) == 0:
        return k_prediction[0] 
    if k_prediction[0] not in duplication: # 最接近的预测值不再重复出现的
        top_dupli = tmp[ min([ tmp.index(d) for d in duplication]) ]# 找出重复预测值中接近程度最高的值
        count = sum([ 1 for i in k_prediction if i==top_dupli]) # 统计较高相似度的重复预测值出现次数
        if count >= int(2/3 * k): # 如果较高相似度的重复预测值出现次数 >= 预测值总数的2/3，则取该预测值
            return top_dupli    
    return k_prediction[0]

In [14]:
def RG_kNN_classifier(feature_set_train, label_set_train, feature_set_test, k=3):
    '''
    所有的feature集合中的图像矩阵需要reshape成一维数组（传入前特征工程需要把图像先resize成20x20再reshape成（400,））
    feature_set_train, label_set_train, feature_set_test都必须是np.array
    '''
    label_set_Kpredict, topK_list = [],[] # 增加topK_list记录最接近的图像编号
    for test_feature in feature_set_test:
        distances = []
        for train_feature in feature_set_train:
            distances.append(
                np.sqrt(np.sum(np.square(test_feature - train_feature)))
            )
        # topK = np.argsort(distances)[-k:][::-1] # [-k:]是提取最后k个（因为argsort是升序排序）,[::-1]则是将其反序
        topK = np.argsort(distances)[:k] # ummmm, 我感觉我昨天写代码脑袋写坏了，应该取距离最小的，我上面那句取了距离最大的k个
        if k != 1:
            label_set_Kpredict.append(
                tuple(label_set_train[list(topK)]) # 保存最接近的k个预测值
            )
            topK_list.append(tuple(topK))
        else:
            label_set_Kpredict.append(
                tuple(label_set_train[list(topK)])[0] # 保存最接近的k个预测值
            )
            topK_list.append(tuple(topK))
    if k==1:
        return label_set_Kpredict, topK_list
    else:
        # k个最接近的值，做一个投票器，利用投票器选出最终预测值
        label_set_single = [ judger(labels) for labels in label_set_Kpredict]
        return label_set_single, label_set_Kpredict, topK_list

## 【设计过程】

解释一下`judger()`函数：    
因为kNN计算的是与目标样本最接近的k个预测值，在`RG_kNN_classifier()`中，预测结果是一个tuple      
结构为 $ (p_1, p_2, \cdots, p_k)$，其中，$p_1$最接近目标样本的预测（距离最小），因此其预测价值应该最大  
所以，为了给出最后的预测值，我写了该`judger()`判决函数。
判决流程为：
+ 从 $ k\_prediction = (p_1, p_2, \cdots, p_k)$ 中找出重复出现的预测值，保存到duplication中
+ 如果 duplication 为空(即不存在重复预测值) 或者 $p_1 \in duplication$ ，则直接返回 预测值 $p_1$，结束流程，否则继续接下来的流程
+ 找出duplication中预测价值最大（也就是序号最小）的值 top_dupli
+ 统计 top_dupli 出现的次数，记作 count
+ 如果 $ count >= [ 2/3 \cdot k]$，则返回 top_dupli，否则依旧返回 $p_1$ (其中[]为取整)
至于为啥是 2/3， 是我设置的值，可以自行调整     
举例：
+ (3, 2, 1, 1, 1) 返回 1
+ (1, 2, 3, 4) 返回 1
+ (1, 2, 1, 1, 2) 返回 1

### 先用cv读取图像看看效果

In [15]:
# 项目路径配置
ROOT_DIR = os.getcwd()
IMG_DTR = os.path.join(ROOT_DIR, 'images')
target_imgs = [ os.path.join(IMG_DTR, img_name) for img_name in os.listdir(IMG_DTR)] 
target_imgs.sort() # 升序，注意不能 listA = listA.sort(), sort没有返回值
target = target_imgs[4]
img = cv.imread(target, cv.IMREAD_UNCHANGED) # 以原始格式读入图片
img.shape 

(117, 220, 3)

这里读的图是 验证码.jpg 其尺寸为 117x220 （长x宽），色彩空间为 RGB    
看img矩阵的shape为 (117,220,3)，其中3为像素模式，cv顺序为BGR

## 【主要数据结构】

## 【代码调试过程以及实验结果数据】

## 【讨论与心得】

## 【实验中遇到的问题和解决方法】

## 【参考资料】